# Flight Data Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [101]:
from pyarrow.flight import FlightClient, Ticket, FlightCallOptions
import json
import time
from IPython.display import display, HTML

The class `query_timer` provides a simple stop watch function for query performance testing. This will be utilised later within the notebook.

In [102]:
class query_timer:
    def __init__(self):
        self.start_time = time.time()
        self.end_time = None
        self.elapsed_time = None

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        self.elapsed_time = self.end_time - self.start_time
        print('Execution time:', self.elapsed_time, 'seconds')
        return self.elapsed_time


Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [103]:
token = 'apiv1_ymfGzIFSC6/cgb0/TLf5waGikqht4ddrAkpfklA2byCJ6qFrf4tc6dSJVwSHGntD'
host = 'b0c7cce5-8dbc-428e-98c6-7f996fb96467.a.influxdb.io'    
client = FlightClient(f"grpc+tls://{host}:443")
  
    
    # create an authorization header
options = FlightCallOptions(headers=[(b"authorization",f"Bearer {token}".encode('utf-8'))])

The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

In [104]:
# This functionality will be wrapped in the Python client library, 
# but it's good to see how it works
def query_tools(database="", query="", language="sql"):
    # create a flight client pointing to the InfluxDB
    # create a ticket
    ticket_data = {
    "database": database,
    "sql_query": query,
    "query_type": language}
    
    ticket_bytes = json.dumps(ticket_data)
    ticket = Ticket(ticket_bytes)
    
    # execute the query and return all the data
    flight_reader = client.do_get(ticket, options)

    # use read_all() to get all of the data as an Arrow table
    # there are other functions to iterate through rows or read only parts of the data
    # which is useful if you have huge data sets
    return flight_reader.read_all()

Return all columns from the table flight for the past 3 days.

In [105]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SELECT * FROM \"flight\" WHERE time >= now() -  interval '1 hour'")
df = table.to_pandas()
display(df)

#TODO add number of rows returned if required.

Execution time: 0.5603551864624023 seconds


,baro_altitude,callsign,category,geo_altitude,icao24,last_contact,latitude,longitude,on_ground,origin_country,position_source,sensors,spi,squawk,time,time_position,true_track,velocity,vertical_rate


Count the number of unique tags

In [106]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SELECT count(distinct(icao24)) as \"icao24\", count(distinct(origin_country)) as \"origin_country\" FROM \"flight\" WHERE time >= now() - interval '3 day'")
df = table.to_pandas()
display(df)

Execution time: 0.1254589557647705 seconds


,icao24,origin_country
0,12821,103


# InfluxQL based query

In [107]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SELECT * FROM flight WHERE time > now() - 4d limit 100000", language="influxql")
df = table.to_pandas()
display(df)

Execution time: 2.090500831604004 seconds


,iox::measurement,time,baro_altitude,callsign,category,geo_altitude,icao24,last_contact,latitude,longitude,on_ground,origin_country,position_source,sensors,spi,squawk,time_position,true_track,velocity,vertical_rate
0,flight,2023-05-19 13:30:47.119083,7924.80,RJE43,0,8282.94,010042,1684528238,39.1048,-84.4277,True,Egypt,0,None,False,4770,1684528238,67.50,0.00,0.00
1,flight,2023-05-19 13:30:47.119083,4191.00,RPB7073,1,4450.08,0c2045,1684528240,4.9475,-74.5097,False,Panama,0,None,False,1701,1684528240,109.67,143.68,-2.60
2,flight,2023-05-19 13:30:47.119083,5692.14,RPB7275,1,6050.28,0c204f,1684528240,5.2631,-74.7500,False,Panama,0,None,False,1000,1684528240,134.03,171.72,-5.20
3,flight,2023-05-19 13:30:47.119083,11582.40,RPB7411,4,12374.88,0c2051,1684528240,-6.8354,-76.3945,False,Panama,0,None,False,1676,1684528240,194.07,222.22,0.00
4,flight,2023-05-19 13:30:47.119083,10668.00,RPB7280,1,11109.96,0c2059,1684528035,4.7104,-74.1515,True,Panama,0,None,False,5657,1684528035,306.56,3.09,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,flight,2023-05-19 13:36:07.148814,6111.24,ANA844,6,6370.32,86ee7e,1684528240,34.4218,139.1496,False,Japan,0,None,False,6105,1684528240,44.20,208.09,-2.93
99996,flight,2023-05-19 13:36:07.148814,11887.20,ANA898,6,12321.54,86eec2,1684528240,33.7520,136.6386,False,Japan,0,None,False,4357,1684528240,75.38,281.25,-0.33
99997,flight,2023-05-19 13:36:07.148814,152.40,ANA850,0,152.40,86ef6c,1684528011,35.5247,139.7940,False,Japan,0,None,False,6115,1684528011,330.06,74.21,-3.90
99998,flight,2023-05-19 13:36:07.148814,11582.40,AIQ395,4,12336.78,880850,1684528240,-0.1859,104.1901,False,Thailand,0,None,False,7133,1684528240,352.91,229.14,0.00


# Schema Navigation

In this section you can find some examples for schema navigation. 

In [108]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SHOW FIELD KEYS", language="influxql")
df = table.to_pandas()
display(df)

Execution time: 0.10583209991455078 seconds


,iox::measurement,fieldKey,fieldType
0,flight,baro_altitude,float
1,flight,callsign,string
2,flight,category,integer
3,flight,geo_altitude,float
4,flight,last_contact,integer
5,flight,latitude,float
6,flight,longitude,float
7,flight,on_ground,boolean
8,flight,position_source,integer
9,flight,sensors,string


In [109]:
with query_timer() as timer:
    table = query_tools(database="flightdemo",query="SHOW TAG KEYS", language="influxql")
df = table.to_pandas()
display(df)

Execution time: 0.1018378734588623 seconds


,iox::measurement,tagKey
0,flight,icao24
1,flight,origin_country
